In [ ]:
# api 설치
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

from fastapi import FastAPI
from pydantic import BaseModel
from contextlib import asynccontextmanager
from module.model import load_models, predict
from module.chatbot import get_chatbot_chain
from module.wordcount import count_tokens

c:\Users\kase\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# lifespan으로 모델 로드 관리
@asynccontextmanager
async def lifespan(app: FastAPI):
    global models, tokenizers, device
    models, tokenizers, device = load_models()
    yield
    models.clear()
    tokenizers.clear()

In [ ]:
app = FastAPI(lifespan=lifespan)

In [ ]:

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    reply: str

@app.get("/api/v1/chatbot", response_model=ChatResponse)
def chat(req: ChatRequest):
    response = get_chatbot_chain.run(input=req.message)
    return ChatResponse(reply=response)

- 응답예시{json 형식}  
{
  "reply": "오늘 하루 많이 힘드셨겠어요."
}

In [ ]:
class TextInput(BaseModel):
    text: str

@app.get("/api/v1/channels/{channelId}/reports/frequency")
def analyze_text(channelId: str, data: TextInput):
    # 1) 단어 카운트
    counter = count_tokens(data.text)
    token_freqs = [{"token": tok, "count": cnt} for tok, cnt in counter.most_common()]

    # 2) 감지 모델 예측
    results = {}
    for disease in ["depression", "anxiety"]:
        results[disease] = predict(data.text, models[disease], tokenizers[disease], device)

    # 3) 통합 Response
    return {
        "channelId": channelId,
        "input": data.text,
        "tokens": token_freqs,
        "predictions": results
    }


- 응답예시(json형식)  
""" {
  "channelId": "123",  
  "input": "오늘은 기분이 우울하고 불안하다.",  
  "tokens": [  
    {"token": "기분", "count": 1},  
    {"token": "우울하고", "count": 1},  
    {"token": "불안하다", "count": 1}  
  ],  
  "predictions": {  
    "depression": 2,  
    "anxiety": 1  
  }  
} """

In [ ]:
# 직동확인 부분(최종 확인후 주석처리 혹은 삭제할 것)
@app.get("/")
def root():
    return {"message": "KLUEBERT API is running", "models": ["depression", "anxiety"]}